In [1]:
import pinocchio as pin
import os
from pinocchio.visualize import MeshcatVisualizer
import meshcat.geometry as g
import meshcat.transformations as tf

# Define the path to your URDF file
URDF_PATH = "/home/medusa/bravo7_compliance/reach-bravo-7/bpl_bravo_description/urdf/bravo_7_noFT_pinocchio.urdf"

# Define the root directory of your robot package.
# This should be the directory that contains the 'bpl_bravo_description' folder.
# For example, if your URDF is at '/home/medusa/bravo7_compliance/reach-bravo-7/bpl_bravo_description/urdf/...',
# then the package directory is '/home/medusa/bravo7_compliance/reach-bravo-7/'.
PACKAGE_DIR = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(URDF_PATH))))

# The 'bpl_bravo_description' is the package name.
PACKAGE_NAME = "bpl_bravo_description"

# Load the URDF model, specifying the package directory
try:
    model, collision_model, visual_model = pin.buildModelsFromUrdf(
        URDF_PATH, package_dirs=[PACKAGE_DIR], meshLoader=pin.MeshLoader()
    )

    # Initialize the Meshcat visualizer
    from pinocchio.visualize import MeshcatVisualizer
    viz = MeshcatVisualizer(model, collision_model, visual_model)
    viz.initViewer(open=True)
    viz.loadViewerModel()

    # Display the robot in a neutral configuration
    q0 = pin.neutral(model)
    viz.display(q0)
    
except Exception as e:
    print(f"An error occurred: {e}")

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [2]:
model_amir = pin.buildModelFromUrdf("/home/medusa/bravo7_compliance/reach-bravo-7/bpl_bravo_description/urdf/bravo_7_amir_2.urdf")
data_amir = model_amir.createData()
tool_link = "EE"

In [3]:
import numpy as np
STORE = np.array([4.904, 2.857, 0.05, 3.387, 0.066, 0.2304])
HOME = np.array([4.904, 2.592, 0.549, 4.904, 1.570, 0.0])

update.go:85: cannot change mount namespace according to change mount (/var/lib/snapd/hostfs/usr/local/share/doc /usr/local/share/doc none bind,ro 0 0): cannot write to "/var/lib/snapd/hostfs/usr/local/share/doc" because it would affect the host in "/var/lib/snapd"
update.go:85: cannot change mount namespace according to change mount (/var/lib/snapd/hostfs/usr/share/gimp/2.0/help /usr/share/gimp/2.0/help none bind,ro 0 0): cannot write to "/var/lib/snapd/hostfs/usr/share/gimp/2.0/help" because it would affect the host in "/var/lib/snapd"
update.go:85: cannot change mount namespace according to change mount (/var/lib/snapd/hostfs/usr/share/sphinx_rtd_theme /usr/share/sphinx_rtd_theme none bind,ro 0 0): cannot write to "/var/lib/snapd/hostfs/usr/share/sphinx_rtd_theme" because it would affect the host in "/var/lib/snapd"
update.go:85: cannot change mount namespace according to change mount (/var/lib/snapd/hostfs/usr/share/xubuntu-docs /usr/share/xubuntu-docs none bind,ro 0 0): cannot wri

[GFX1-]: More than 1 GPU from same vendor detected via PCI, cannot deduce device



In [4]:
q_rand = pin.randomConfiguration(model)
viz.display(HOME)

In [5]:
type(q_rand)

numpy.ndarray

In [6]:
g.MeshLambertMaterial(
    color=0x00ff00,   # green
    transparent=True,
    opacity=0.3
)


In [7]:
frame_id_ee = model.getFrameId("EE") 
frame_id_ee = 43

In [8]:
# 1) update kinematics
pin.forwardKinematics(model_amir, data_amir, HOME)
pin.updateFramePlacements(model_amir, data_amir)
# 4) Frame Jacobian (6 x nv) in chosen reference frame
J = pin.computeFrameJacobian(model_amir, data_amir, HOME, frame_id_ee, pin.ReferenceFrame.LOCAL)  # shape (6 x nv)
rot = data_amir.oMf[frame_id_ee].rotation 
position = data_amir.oMf[frame_id_ee].translation # shape (3 x 1)
poseT = data_amir.oMf[frame_id_ee].homogeneous # shape (4 x 4)



In [9]:
# --- Manipulability ellipsoids ---
# Motion ellipsoid: v = J*qdot
JJT = J[:3, :] @ J[:3, :].T   # Linear velocity manipulability (3x3)
eigvals, eigvecs = np.linalg.eigh(JJT)

# Force ellipsoid: f = (J^T)^-1 tau
JJT_inv = np.linalg.inv(JJT)
eigvals_f, eigvecs_f = np.linalg.eigh(JJT_inv)

In [10]:
def draw_ellipsoid(viz, name, eigvecs, eigvals, pose, color=0x00ff00, opacity=0.3, scale_factor=0.05):
    radii = np.sqrt(np.maximum(eigvals, 0))
    scales = scale_factor * radii

    T = pose
    T[:3,:3] = eigvecs

    material = g.MeshLambertMaterial(color=color, transparent=True, opacity=opacity)
    viz.viewer[name].set_object(g.Ellipsoid(scales), material)
    viz.viewer[name].set_transform(T)


In [11]:
# Show at end-effector
viz.viewer["world"]["motion_ellipsoid"].set_transform(poseT)
draw_ellipsoid(viz, "world/motion_ellipsoid", eigvecs, eigvals, pose=poseT, color=0x2609DE, opacity=0.9)


In [12]:

viz.viewer["world"]["force_ellipsoid"].set_transform(poseT)
draw_ellipsoid(viz, "world/force_ellipsoid", eigvecs_f, eigvals_f,scale_factor=0.02, pose=poseT, color=0xff0000)

In [16]:
import ipywidgets as widgets
from ipywidgets import interact

In [40]:
# --- interactive sliders ---
interact(update,
         theta1=widgets.FloatSlider(min=-1.5, max=1.5, step=0.1, value=0.0),
         theta2=widgets.FloatSlider(min=-1.5, max=1.5, step=0.1, value=0.0),
         theta3=widgets.FloatSlider(min=-1.5, max=1.5, step=0.1, value=0.0),
         scale_factor=widgets.FloatSlider(min=0.01, max=0.2, step=0.01, value=0.05))

NameError: name 'update' is not defined

In [50]:
model_amir = pin.buildModelFromUrdf("/home/medusa/bravo7_compliance/reach-bravo-7/bpl_bravo_description/urdf/bravo_7_amir_2.urdf")
data_amir = model_amir.createData()
tool_link = "EE"
HOME = np.array([4.904, 2.592, 0.549, 4.904, 1.570, 0.0])

def update(theta1=4.904, theta2=2.592, theta3=0.549, theta4=4.904, theta5=1.570, theta6=0.0, scale_factor=0.05):
    # update joint configuration (example: 3 DoF robot, extend as needed)
    q = pin.neutral(model_amir)
    q[0] = theta1
    q[1] = theta2
    q[2] = theta3
    q[3] = theta4
    q[4] = theta5
    q[5] = theta6

    viz.display(q)
    pin.forwardKinematics(model_amir, data_amir, q)
    pin.updateFramePlacements(model_amir, data_amir)

    J = pin.computeFrameJacobian(model_amir, data_amir, q, 43, pin.ReferenceFrame.LOCAL) 
    Jlin = J[:3,:]

    # motion ellipsoid
    JJT = Jlin @ Jlin.T
    eigvals, eigvecs = np.linalg.eigh(JJT)
    placement = data_amir.oMf[43].homogeneous
    draw_ellipsoid(viz, "world/motion_ellipsoid", eigvecs, eigvals, placement,
                   color=0x2609DE, opacity=0.9, scale_factor=scale_factor)

    # force ellipsoid
    JJT_inv = np.linalg.inv(JJT)
    eigvals_f, eigvecs_f = np.linalg.eigh(JJT_inv)
    draw_ellipsoid(viz, "world/force_ellipsoid", eigvecs_f, eigvals_f, placement,
                   color=0xff0000, opacity=0.4, scale_factor=0.02)
    print("Updated configuration:", q)

# --- interactive sliders ---
interact(update,
         theta1=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=4.904),
         theta2=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=2.592),
         theta3=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=0.549),
         theta4=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=4.904),
         theta5=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=1.570),
         theta6=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=0.0),
         scale_factor=widgets.FloatSlider(min=0.01, max=0.2, step=0.01, value=0.05))

interactive(children=(FloatSlider(value=4.904, description='theta1', max=10.0, min=-5.0), FloatSlider(value=2.…

<function __main__.update(theta1=4.904, theta2=2.592, theta3=0.549, theta4=4.904, theta5=1.57, theta6=0.0, scale_factor=0.05)>

In [35]:
def update_xyz_local(dx=0.0, dy=0.0, dz=0.0, scale_factor=0.05):
    # start from HOME configuration
    q = pin.neutral(model_amir)
    q[:6] = HOME  

    # forward kinematics
    pin.forwardKinematics(model_amir, data_amir, q)
    pin.updateFramePlacements(model_amir, data_amir)

    frame_id = model_amir.getFrameId(tool_link)

    # compute Jacobian in LOCAL frame
    J = pin.computeFrameJacobian(model_amir, data_amir, q, frame_id, pin.ReferenceFrame.WORLD)
    Jlin = J[:3, :]

    # desired displacement in LOCAL frame
    dx_local = np.array([dx, dy, dz])

    # compute joint update via pseudoinverse
    dq = np.linalg.pinv(Jlin) @ dx_local

    # apply to current q
    q[:6] += dq

    # display robot
    viz.display(q)
    pin.forwardKinematics(model_amir, data_amir, q)
    pin.updateFramePlacements(model_amir, data_amir)

    # get placement of EE
    placement = np.eye(4)

    # motion ellipsoid
    Jlin = J[:3, :]
    JJT = Jlin @ Jlin.T
    eigvals, eigvecs = np.linalg.eigh(JJT)
    draw_ellipsoid(viz, "world/motion_ellipsoid", eigvecs, eigvals, placement,
                   color=0x2609DE, opacity=0.9, scale_factor=scale_factor)

    # force ellipsoid
    JJT_inv = np.linalg.inv(JJT)
    eigvals_f, eigvecs_f = np.linalg.eigh(JJT_inv)
    draw_ellipsoid(viz, "world/force_ellipsoid", eigvecs_f, eigvals_f, placement,
                   color=0xff0000, opacity=0.4, scale_factor=0.02)

    print("Applied local displacement:", dx_local)
    print("New configuration:", q)

In [36]:
interact(update_xyz_local,
         dx=widgets.FloatSlider(min=-0.1, max=0.1, step=0.01, value=0.0),
         dy=widgets.FloatSlider(min=-0.1, max=0.1, step=0.01, value=0.0),
         dz=widgets.FloatSlider(min=-0.1, max=0.1, step=0.01, value=0.0),
         scale_factor=widgets.FloatSlider(min=0.01, max=0.2, step=0.01, value=0.05))

interactive(children=(FloatSlider(value=0.0, description='dx', max=0.1, min=-0.1, step=0.01), FloatSlider(valu…

<function __main__.update_xyz_local(dx=0.0, dy=0.0, dz=0.0, scale_factor=0.05)>

In [13]:
# global variable to store current configuration
q_current = pin.neutral(model_amir)
q_current[:6] = HOME  

def update_pose(dx=0.0, dy=0.0, dz=0.0, rx=0.0, ry=0.0, rz=0.0, scale_factor=0.05):
    global q_current

    frame_id = model_amir.getFrameId(tool_link)
    frame_id = 43

    # compute Jacobian at current configuration
    pin.forwardKinematics(model_amir, data_amir, q_current)
    pin.updateFramePlacements(model_amir, data_amir)
    J = pin.computeFrameJacobian(model_amir, data_amir, q_current, frame_id, pin.ReferenceFrame.LOCAL)

    # build displacement vector (linear + angular, in LOCAL frame)
    dx_local = np.array([dx, dy, dz, rx, ry, rz])

    # compute joint update
    dq = np.linalg.pinv(J) @ dx_local

    # update q_current
    q_current[:6] += dq

    # display robot
    viz.display(q_current)
    pin.forwardKinematics(model_amir, data_amir, q_current)
    pin.updateFramePlacements(model_amir, data_amir)

    # placement of EE
    placement = np.eye(4)
    placement[1, 3] = 0.5
    
    # motion ellipsoid
    Jlin = J[:3, :]   # use only linear velocity part
    JJT = Jlin @ Jlin.T
    eigvals, eigvecs = np.linalg.eigh(JJT)
    eigvals = np.maximum(eigvals, 1e-12)      # numerical safety
    axis_lengths = np.sqrt(eigvals)           # take sqrt for axes
    #placement = data_amir.oMf[frame_id].homogeneous
    draw_ellipsoid(viz, "world/motion_ellipsoid", eigvecs, axis_lengths, placement,
                color=0x2609DE, opacity=0.9, scale_factor=scale_factor)

    # force ellipsoid
    JJT_inv = np.linalg.inv(JJT)
    eigvals_f, eigvecs_f = np.linalg.eigh(JJT_inv)
    eigvals_f = np.maximum(eigvals_f, 1e-12)
    axis_lengths_f = np.sqrt(eigvals_f)
    draw_ellipsoid(viz, "world/force_ellipsoid", eigvecs_f, axis_lengths_f, placement,
                color=0xff0000, opacity=0.4, scale_factor=0.02)

    print("Applied local displacement:", dx_local)
    print("New configuration:", q_current)

In [58]:
slider_layout = widgets.Layout(width='80%')  # 600px wide
interact(update_pose,
         dx=widgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, layout=slider_layout),
         dy=widgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, layout=slider_layout),
         dz=widgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, layout=slider_layout),
         rx=widgets.FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.0, layout=slider_layout),
         ry=widgets.FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.0, layout=slider_layout),
         rz=widgets.FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.0, layout=slider_layout))


interactive(children=(FloatSlider(value=0.0, description='dx', layout=Layout(width='80%'), max=0.3, min=-0.3, …

<function __main__.update_pose(dx=0.0, dy=0.0, dz=0.0, rx=0.0, ry=0.0, rz=0.0, scale_factor=0.05)>

In [44]:
import numpy as np
import pinocchio as pin
import meshcat.geometry as g

# Draw with U (principal directions) and radii already computed (no extra sqrt inside)
def draw_ellipsoid_from_svd(viz, path, U, radii, placement, color, opacity=0.4, scale=0.05):
    radii = np.asarray(radii, dtype=float)
    radii = np.maximum(radii, 1e-12)  # clamp to avoid zeros
    T = placement.copy()
    T[:3, :3] = T[:3, :3] @ U
    material = g.MeshLambertMaterial(color=color, transparent=True, opacity=opacity)
    viz.viewer[path].set_object(g.Ellipsoid(scale * radii), material)
    viz.viewer[path].set_transform(T)

# Global config (keep your q_current logic)
q_current = pin.neutral(model_amir)
q_current[:6] = HOME

def update_pose(dx=0.0, dy=0.0, dz=0.0, rx=0.0, ry=0.0, rz=0.0, scale_factor=0.05):
    global q_current

    # Use the real EE frame id; do NOT overwrite it with a hard-coded number
    frame_id = model_amir.getFrameId(tool_link)

    # FK/Jacobian at current config
    pin.forwardKinematics(model_amir, data_amir, q_current)
    pin.updateFramePlacements(model_amir, data_amir)
    J = pin.computeFrameJacobian(model_amir, data_amir, q_current, frame_id, pin.LOCAL)
    Jlin = J[:3, :]

    # Small damped IK step (more stable than raw pinv)
    dx_local = np.array([dx, dy, dz, rx, ry, rz])
    mu = 1e-4
    JJt = J @ J.T
    dq = J.T @ np.linalg.solve(JJt + (mu**2)*np.eye(6), dx_local)

    # Update only valid DOFs
    q_current[:len(dq)] += dq[:len(q_current)]

    # Update kinematics again after the move
    viz.display(q_current)
    pin.forwardKinematics(model_amir, data_amir, q_current)
    pin.updateFramePlacements(model_amir, data_amir)

    # ⬇️ SVD-based manipulability (clean and aligned)
    U, s, Vt = np.linalg.svd(Jlin, full_matrices=False)
    eps = 1e-12
    s = np.maximum(s, eps)
    motion_radii = s
    force_radii  = 1.0 / s

    # Place ellipsoids at the actual EE pose
    placement = data_amir.oMf[frame_id].homogeneous

    # Colors: motion = BLUE, force = RED (keep them consistent)
    draw_ellipsoid_from_svd(viz, "world/motion_ellipsoid", U, motion_radii, placement,
                            color=0x2609DE, opacity=0.45, scale=0.2)
    draw_ellipsoid_from_svd(viz, "world/force_ellipsoid", U, force_radii, placement,
                            color=0xFF0000, opacity=0.35, scale=0.02)

    # Quick sanity print
    print("singular values σ (motion radii):", s)
    print("1/σ (force radii):", 1.0/s)


In [45]:
slider_layout = widgets.Layout(width='80%')  # 600px wide
interact(update_pose,
         dx=widgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, layout=slider_layout),
         dy=widgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, layout=slider_layout),
         dz=widgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, layout=slider_layout),
         rx=widgets.FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.0, layout=slider_layout),
         ry=widgets.FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.0, layout=slider_layout),
         rz=widgets.FloatSlider(min=-0.2, max=0.2, step=0.01, value=0.0, layout=slider_layout))


interactive(children=(FloatSlider(value=0.0, description='dx', layout=Layout(width='80%'), max=0.3, min=-0.3, …

<function __main__.update_pose(dx=0.0, dy=0.0, dz=0.0, rx=0.0, ry=0.0, rz=0.0, scale_factor=0.05)>

In [53]:
def draw_joint_configuration_and_ellipsoids(joint_configuration):

    # Use the real EE frame id; do NOT overwrite it with a hard-coded number
    frame_id = model_amir.getFrameId(tool_link)

    # FK/Jacobian at current config
    pin.forwardKinematics(model_amir, data_amir, joint_configuration)
    pin.updateFramePlacements(model_amir, data_amir)
    J = pin.computeFrameJacobian(model_amir, data_amir, joint_configuration, frame_id, pin.LOCAL)
    Jlin = J[:3, :]


    mu = 1e-4
    JJt = J @ J.T


    # Update kinematics again after the move
    viz.display(joint_configuration)
    pin.forwardKinematics(model_amir, data_amir, joint_configuration)
    pin.updateFramePlacements(model_amir, data_amir)

    # ⬇️ SVD-based manipulability (clean and aligned)
    U, s, Vt = np.linalg.svd(Jlin, full_matrices=False)
    eps = 1e-12
    s = np.maximum(s, eps)
    motion_radii = s
    force_radii  = 1.0 / s

    # Place ellipsoids at the actual EE pose
    placement = data_amir.oMf[frame_id].homogeneous

    # Colors: motion = BLUE, force = RED (keep them consistent)
    draw_ellipsoid_from_svd(viz, "world/motion_ellipsoid", U, motion_radii, placement,
                            color=0x2609DE, opacity=0.45, scale=0.2)
    draw_ellipsoid_from_svd(viz, "world/force_ellipsoid", U, force_radii, placement,
                            color=0xFF0000, opacity=0.35, scale=0.02)

    # Quick sanity print
    print("singular values σ (motion radii):", s)
    print("1/σ (force radii):", 1.0/s)

def update_joint_position(theta1=4.904, theta2=2.592, theta3=0.549, theta4=4.904, theta5=1.570, theta6=0.0, scale_factor=0.05):
    # update joint configuration (example: 3 DoF robot, extend as needed)
    q = pin.neutral(model_amir)
    q[0] = theta1
    q[1] = theta2
    q[2] = theta3
    q[3] = theta4
    q[4] = theta5
    q[5] = theta6

    viz.display(q)
    draw_joint_configuration_and_ellipsoids(q)


# --- interactive sliders ---
interact(update_joint_position,
         theta1=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=4.904),
         theta2=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=2.592),
         theta3=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=0.549),
         theta4=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=4.904),
         theta5=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=1.570),
         theta6=widgets.FloatSlider(min=-5.0, max=10.0, step=0.1, value=0.0),
         scale_factor=widgets.FloatSlider(min=0.01, max=0.2, step=0.01, value=0.05))

interactive(children=(FloatSlider(value=4.904, description='theta1', max=10.0, min=-5.0), FloatSlider(value=2.…

<function __main__.update_joint_position(theta1=4.904, theta2=2.592, theta3=0.549, theta4=4.904, theta5=1.57, theta6=0.0, scale_factor=0.05)>

In [51]:
configuration = np.array([3.0, 2.592, 0.8, 3.10, 1.6, 0.0])
draw_joint_configuration_and_ellipsoids(configuration)

singular values σ (motion radii): [0.6453142  0.60070919 0.1744701 ]
1/σ (force radii): [1.54963271 1.66469903 5.73164116]
